In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy
from ALLSampler_Sentence import SamplingSimulator, ModelSamplingSimulator

In [2]:
logger.remove()
logger.add(sys.stderr, level='DEBUG')

1

## Initiate Brat Reader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read Brat annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


## Define sampling function

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

## CRF Wrapper

In [10]:
from CRFWrapper_Sentence import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel


In [11]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Dosage', 'Reason', 'Route', 'Drug', 'Strength', 'Frequency', 'Form', 'Duration', 'ADE'}


In [12]:
crf_model=CRFModel(anno_types=annos)

## Convert training and testing docs into sentence level dataframe

In [13]:
_, train_df=convert_docs(train_docs, anno_types=annos)

In [14]:
_, test_df=convert_docs(test_docs, anno_types=annos)

In [ ]:
train_df

In [16]:
train_df.label.unique()

array(['O', 'B-Drug', 'B-Route', 'B-Reason', 'I-Reason', 'B-Strength',
       'I-Strength', 'B-Frequency', 'B-Duration', 'I-Duration', 'B-Form',
       'B-Dosage', 'I-Dosage', 'B-ADE', 'I-Frequency', 'I-Drug', 'I-Form',
       'I-Route', 'I-ADE'], dtype=object)

## Bootstrap 3 different initials runs

In [17]:
boostrap_runs=3
total_round=10
logger.remove()
logger.add(sys.stderr, level='INFO')

2

In [18]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [19]:
%%time
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos, topNUncertainToken=2)
    m_simulator=ModelSamplingSimulator(total_sents=train_df, 
                                   total_round=10, 
                                   modelWrapper=crf_model, 
                                   eval_sents=test_df, 
                                   init_seed=seed,
                                   sample_all_on_last_round=False)  
    m_simulator.num_per_round=200
    scores=m_simulator.simulate_rounds(boostrap_times=500)
    all_scores.append(scores) 

2024-07-15 16:44:10.702 | INFO     | __main__:<module>:3 - start run 0.
2024-07-15 16:44:10.823 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 0.
2024-07-15 16:44:11.186 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 200, remaining sentences: 51598
2024-07-15 16:53:43.880 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 1.
2024-07-15 16:54:47.678 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 400, remaining sentences: 51398
2024-07-15 17:05:01.810 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 2.
2024-07-15 17:05:55.175 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 600, remaining sentences: 51198
2024-07-15 17:14:17.323 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 3.
2024-07-15 17:15:10.142 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 800, remaining sentences: 50998
2024-07-15 1

CPU times: total: 4h 40min 58s
Wall time: 4h 41min 22s


In [20]:
joblib.dump(all_scores, r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_500bootstrap.joblib')


['../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_500bootstrap.joblib']

  ## Debug

In [21]:
model=m_simulator.modelWrapper

In [22]:
sampled=m_simulator.sampled

In [23]:
sampled.shape

(34901, 5)

In [77]:
remaining=m_simulator.remaining
remaining.shape

(898005, 5)

In [130]:
original_idx=[]
X=[]
y=[]
#iterate through group: idx is the certain sentenceID, sdf is the sub df under certain sentence ID
k=0
for idx, sdf in remaining.groupby('sentence_id'): 
    #append sentence ID to list; sdf also has 4 columns:sentence_id	doc_name	token	label
    print(idx)
    original_idx.append(sdf.index[0]) 
    print()
    x=sent2features(list(sdf['token']))
    X.append(sent2features(list(sdf['token'])))
    y.append(list(sdf['label']))
    k+=1

In [140]:
X,y =__df2features(remaining)

[3, 32, 78, 50, 13, 18, 22, 12, 28, 14]


In [141]:
print([len(x) for x in X[:10]])

[3, 32, 78, 50, 13, 18, 22, 12, 28, 14]


In [131]:
print([len(x) for x in X[:10]])

[3, 32, 78, 50, 13, 18, 22, 12, 28, 14]


In [80]:
len(remaining['sentence_id'].unique())

49798

In [28]:
yp=model.crf.predict_marginals(X)

In [53]:
remaining[:20]

,sentence_id,doc_name,token,label,sentence
0,0,100035.txt,Admission,O,Admission Date:
1,0,100035.txt,Date,O,Admission Date:
2,0,100035.txt,:,O,Admission Date:
3,1,100035.txt,[,O,[**2115-2-22**] Discharge Date: ...
4,1,100035.txt,*,O,[**2115-2-22**] Discharge Date: ...
5,1,100035.txt,*,O,[**2115-2-22**] Discharge Date: ...
6,1,100035.txt,2115,O,[**2115-2-22**] Discharge Date: ...
7,1,100035.txt,-,O,[**2115-2-22**] Discharge Date: ...
8,1,100035.txt,2,O,[**2115-2-22**] Discharge Date: ...
9,1,100035.txt,-,O,[**2115-2-22**] Discharge Date: ...


In [74]:
len(X[2])

3

In [60]:
X[4]

[{'bias': 1.0,
  'word.lower()': 'admission',
  'word[-3:]': 'ion',
  'word[-2:]': 'on',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PROPN',
  'postag[:2]': 'PR',
  'BOS': True,
  '+1:word.lower()': 'date',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:postag': 'PROPN',
  '+1:postag[:2]': 'PR'},
 {'bias': 1.0,
  'word.lower()': 'date',
  'word[-3:]': 'ate',
  'word[-2:]': 'te',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PROPN',
  'postag[:2]': 'PR',
  '-1:word.lower()': 'admission',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'PROPN',
  '-1:postag[:2]': 'PR',
  '+1:word.lower()': ':',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'PUNCT',
  '+1:postag[:2]': 'PU'},
 {'bias': 1.0,
  'word.lower()': ':',
  'word[-3:]': ':',
  'word[-2:]': ':',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()'

In [52]:
sent_logits=yp[11]
len(sent_logits)

3

In [32]:
for w in sent_logits:
    top2=sorted(w.values())

NameError: name 'topNUncertainToken' is not defined

In [41]:
model.single_sent_certainty(sent_logits)

0.9985449700133505

In [42]:
certainty=[]
topNUncertainToken=5
for w in sent_logits:
    # for each word, find the difference between the probabilities of top two uncertain labels.
    top2=sorted(w.values())[-2:]
    certainty.append(top2[1]-top2[0])

In [43]:
certainty

[0.9968102443889114, 0.9988639574730318, 0.9999607081781082]

In [46]:
len(sent_logits)

3